### Objective

Wallapop is a Spanish second-hand resale site (similar to Gumtree or Facebook Marketplace) where users can sell items.

The goal of this project will be to scrape data from Wallapop in order to retrieve information of search results for different types of bicicyle results.

### Criteria

- Initialize a Chrome web driver and access the Wallapop webpage
- Use the searchbox to look for all results matching the keyword “bicicleta” (bike in Spanish)
- Under the different search options provided, only access those entries in the category “Bicicletas” (bikes in Spanish)
- Filter the search results according to the following criteria:
    - 1. Location. Only retrieve results in “España, Barcelona” and narrow down the search to a maximum of “10km”
    - 2. Price. Limit the price to 800€.
    - 3. Subfield. Within the “Bicicletas” field, there are multiple subfields available. Include only results in the “Bicicletas y triciclos” subfield. 
    - 4. Labels. Retrieve only those results that correspond to “Bicicletas de carretera” (road bikes), “Bicicletas plegables” (foldable bikes) or “MTB” (mountain bikes).
    - 5. State. Only include those results that correspond to bikes that are “Nuevo” (New), “Como nuevo” (As good as new) and “En buen estado” (In good condition).
- Considering all the different combinations among the options above ("Bicicletas de carretera" and "Nuevo", "Bicicletas de carretera" and "En buen estado", etc.), retrieve the following information for all the available results (if less than 250) or for the first 250 results (if more than 250): Exclude all results corresponding to advertss.
    - The URL address to the post
    - The URL address to the displayed image
    - The title of the post
    - The price
    - The full description (as shown in the results page)
- Store the retrieved information in a DataFrame called df under the following columns:
    - "Link" (in str form)
    - “Title” (in str/object form)
    - “Description” (in str/object form)
    - “Price” (in float form)
    - “Image” (in str/object form).

### Approach

Create a function for each stage of the scraping process before running them for each of the define subcategory combinations.

Get the list of filtering combinations for the type and state of the item and open a new browser to access the data.

For each browser opened:
* filter the data (general filters with a specific state-type combination)
* try scrolling till we reach the maximum number of allowed results or obtain all the items available from the filter applied
    * if we're not able to load new results in 3 minutes we stop scraping and collect the available data
* create a dataframe with data collected and clean the data, then aggregate for final answer

#### Import required libraries

In [ ]:
import re
import time
import locale
from itertools import product

import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

#### Launch Wallapop home page

In [ ]:
def getPage(driver):
    # Open webpage
    driver.get('https://www.wallapop.com/')
    
    # Wait for cookie options to appear on the page
    cookie_buttons = WebDriverWait(
        driver, 30).until(
            EC.element_to_be_clickable((By.ID, 'onetrust-button-group'))
    )

    # Get the accept cookie button
    accept_cookie = cookie_buttons.find_element(By.ID, 'onetrust-accept-btn-handler')

    # Accept all cookies
    accept_cookie.click()
    
    return driver

#### Add 'bicicleta' search term

In [ ]:
def searchItem(driver):
    # Get the search input
    search = driver.find_element(By.CLASS_NAME, 'Search__input')
    
    # Search for 'bicicleta'
    search.send_keys('bicicleta')
    search.send_keys(Keys.ENTER)
    
    return driver

#### Select all bicicleta filter categories

In [ ]:
def filterData__categories(driver):
    # Get all the filters
    filters = WebDriverWait(
        driver, 30).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'FiltersWrapper__bar'))
    )
    
    # Click the categories (Todas las categorias) filter
    filters.find_element(By.XPATH, "//div[contains(text(), 'Todas las categorías')]").click()
    
    WebDriverWait(driver, 5) # Temporal pause
    
    # Click the Bicicletas to filter the results
    categories = driver.find_element(By.CLASS_NAME, 'CategoriesFilter__grid')
    categories.find_element(By.XPATH, "//span[contains(text(), 'Bicicletas')]").click()
    
    WebDriverWait(driver, 5) # Temporal pause
    
    return driver

#### Filter only for those in Barcelona, Espana

In [ ]:
def filterData__location(driver):
    # Get all the filters
    filters = WebDriverWait(
        driver, 30).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'FiltersWrapper__bar'))
    )
    
    # Click the location (España, Madrid) filter
    filters.find_element(By.XPATH, "//div[contains(text(), 'España')]").click()
    
    # Get location search bar
    search_location = driver.find_element(By.CLASS_NAME, 'LocationFilter__input')
    
    # Search for España, Barcelona
    search_location.clear()
    search_location.send_keys('España, Barcelona')
    
    # Select first option
    search_result = WebDriverWait(
        driver, 30).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'LocationFilter__search'))
    )

    WebDriverWait(
        search_result, 30).until(
            EC.element_to_be_clickable((By.TAG_NAME, 'button'))
    ).click()
    
    # Find the slider and its value
    slider = WebDriverWait(
        driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, "//span[@role='slider']"))
    )
    slider_value = driver.find_element(By.CLASS_NAME, 'ngx-slider-model-value')
    
    # Move slider to reduce search radius
    action = ActionChains(driver)
    action.click_and_hold(slider).move_by_offset(-450, 0).release().perform()
    
    WebDriverWait(driver, 5) # Temporal pause
    
    # Accept changes
    buttons = driver.find_element(By.CLASS_NAME, 'FilterTemplate__actions')
    buttons = buttons.find_elements(By.XPATH, '//tsl-button')

    for button in buttons: 
        if button.text == 'Aplicar':
            button.click()
            break
            
    WebDriverWait(driver, 5) # Temporal pause
            
    return driver

#### Select specific subcategories (road bikes, foldable bikes, mountain bikes)

In [ ]:
def filterData__type(driver, item_type):
    # Get all the filters
    filters = WebDriverWait(
        driver, 30).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'FiltersWrapper__bar'))
    )
    
    # Click the Subcategory (Subcategoría) filter
    filters.find_element(By.XPATH, "//div[contains(text(), 'Subcategoría')]").click()
    
    # Click the bicycles and tricycles categories
    sub_form = WebDriverWait(
        driver, 30).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'MultiSelectFilter__form'))
    )
    subcategories = sub_form.find_elements(By.CLASS_NAME, 'MultiSelectOption')

    for category in subcategories:
        if category.text == 'Bicicletas y triciclos':
            category.click()
            break
            
    # Select Bicicletas de carretera (road bikes), Bicicletas plegables (foldable bikes) & 
    # MTB (mountain bikes) options in the above category
    cat_form = WebDriverWait(
        driver, 30).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'MultiSelectFilter__form'))
    )
    options = cat_form.find_elements(By.CLASS_NAME, 'MultiSelectOption')

    for option in options:
        option.click()
        if option.text in [item_type]:
            option.find_element(By.CLASS_NAME, 'Checkbox__mark').click()
    
    WebDriverWait(driver, 5) # Temporal pause
    
    # Accept changes
    buttons = driver.find_element(By.CLASS_NAME, 'FilterTemplate__actions')
    buttons = buttons.find_elements(By.XPATH, '//tsl-button')

    for button in buttons: 
        if button.text == 'Aplicar':
            button.click()
            break
            
    WebDriverWait(driver, 5) # Temporal pause
    
    # Check if filter was clicked else try filtering again
    try:
        # Get all the filters
        filters = WebDriverWait(
            driver, 30).until(
                EC.element_to_be_clickable((By.CLASS_NAME, 'FiltersWrapper__bar'))
        )

        # Click the Subcategory (Subcategoría) filter
        filters.find_element(By.XPATH, "//div[contains(text(), 'Subcategoría')]")
        
        filterData__type(driver, item_type)
    except:
        pass
            
    return driver

#### Select specific item conditions (new, like new, good condition)

In [ ]:
def filterData__state(driver, item_state):
    # Get all the filters
    filters = WebDriverWait(
        driver, 30).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'FiltersWrapper__bar'))
    )
    
    # Click the State (Estado del producto) filter
    filters.find_element(By.XPATH, "//div[contains(text(), 'Estado del producto')]").click()
    
    # Select Nuevo (New), Como nuevo (As good as new) & En buen estado (In good condition) in above category
    WebDriverWait(
        driver, 30).until(
            EC.element_to_be_clickable((By.TAG_NAME, 'form'))
    )

    options = driver.find_elements(By.CLASS_NAME, 'MultiSelectOption')

    for option in options:    
        label = option.find_element(By.CLASS_NAME, 'MultiSelectOption__label')

        if label.text in [item_state]:
            option.find_element(By.CLASS_NAME, 'Checkbox__mark').click()

    WebDriverWait(driver, 5) # Temporal pause       
    
    # Accept changes
    buttons = driver.find_element(By.CLASS_NAME, 'FilterTemplate__actions')
    buttons = buttons.find_elements(By.XPATH, '//tsl-button')

    for button in buttons: 
        if button.text == 'Aplicar':
            button.click()
            break
            
    WebDriverWait(driver, 5) # Temporal pause
    
    # Check if filter was clicked else try filtering again
    try:
        # Get all the filters
        filters = WebDriverWait(
            driver, 30).until(
                EC.element_to_be_clickable((By.CLASS_NAME, 'FiltersWrapper__bar'))
        )

        # Click the State (Estado del producto) filter
        filters.find_element(By.XPATH, "//div[contains(text(), 'Estado del producto')]")
        
        filterData__state(driver, item_state)
    except:
        pass
            
    return driver

#### Define maximum item price of €800

In [ ]:
def filterData__price(driver):
    # Get all the filters
    filters = WebDriverWait(
        driver, 30).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'FiltersWrapper__bar'))
    )

    # Click the price (Precio) filter
    filters.find_element(By.XPATH, "//div[contains(text(), 'Precio')]").click()

    # Enter the price limit
    max_input = driver.find_element(By.XPATH, "//input[@formcontrolname='max']")
    max_input.clear()
    max_input.send_keys('800')
    
    WebDriverWait(driver, 5) # Temporal pause

    # Accept changes
    buttons = driver.find_element(By.CLASS_NAME, 'FilterTemplate__actions')
    buttons = buttons.find_elements(By.XPATH, '//tsl-button')

    for button in buttons: 
        if button.text == 'Aplicar':
            button.click()
            break
            
    WebDriverWait(driver, 5) # Temporal pause
    
    # Check if filter was clicked else try filtering again
    try:
        # Get all the filters
        filters = WebDriverWait(
            driver, 30).until(
                EC.element_to_be_clickable((By.CLASS_NAME, 'FiltersWrapper__bar'))
        )

        # Click the price (Precio) filter
        filters.find_element(By.XPATH, "//div[contains(text(), 'Precio')]")
        
        filterData__price(driver)
    except:
        pass
            
    return driver

#### Apply all filters

In [ ]:
def filterData(driver, item_type, item_state):
    # Categories
    driver = filterData__categories(driver)
    
    # Location
    driver = filterData__location(driver)
            
    # Type
    driver = filterData__type(driver, item_type)
            
    # State
    driver = filterData__state(driver, item_state)
            
    # Price
    driver = filterData__price(driver)
            
    return driver

#### Load all page items

In [ ]:
def extractData__preprocess(driver):
    # Scroll to end of page if we don't have all the data
    while True:
        try:
            driver.execute_script("window.scrollBy(0, 100);")
            
            footer = WebDriverWait(
                driver, 30).until(
                    EC.visibility_of_element_located((By.CLASS_NAME, 'Footer'))
            )
            break
        except:
            driver.execute_script("window.scrollBy(0, 200);")
            WebDriverWait(driver, 5)
            
    # Check if there are more items and click on the more items button if there are
    try:
        more_button = driver.find_element(By.CLASS_NAME, 'Search__btnContainer')
        more_button.click()
    except:
        pass
    
    # Check the number of items on the page
    item_group = driver.find_element(By.CLASS_NAME, 'ItemCardList')
    num_of_items = len(item_group.find_elements(By.TAG_NAME, 'a'))
    
    start = time.time()

    while True:
        # Previous number of items to track changes
        _ = len(item_group.find_elements(By.TAG_NAME, 'a'))
        
        # Stop retrieval if there are no changes on the page after 3mins
        if (time.time() - start) / 60 > 3: break
            
        try:
        # Stop scrolling if we reach the end of the page 
        # (end of page - footer showing means there are no more results)
            WebDriverWait(
                driver, 5).until(
                    EC.visibility_of_element_located((By.CLASS_NAME, 'Footer'))
                )
            break
        except:
            pass

        # Skip the scrolling part of the loop if the other results are still loading
        try:            
            driver.find_element(By.CLASS_NAME, 'Search__spinner')
            continue
        except:
            pass

        # Scroll
        driver.execute_script("window.scrollBy(0, 500);")

        # Check the number of items on the page after scrolling
        item_group = driver.find_element(By.CLASS_NAME, 'ItemCardList')
        num_of_items = len(item_group.find_elements(By.TAG_NAME, 'a'))

        # Stop scrolling if we have reached the maximum allowed items
        if num_of_items > 250: break

        try:
            # Stop scrolling if we reach the end of the page 
            # (end of page - footer showing means there are no more results)
            WebDriverWait(
                driver, 5).until(
                    EC.visibility_of_element_located((By.CLASS_NAME, 'Footer'))
                )
            break
        except:
            # Scroll
            driver.execute_script("window.scrollBy(0, 500);")
                
    # Get all items listed after all the scrolling we've done
    items = item_group.find_elements(By.TAG_NAME, 'a')
    
    return driver, items[:250] # Return the updated driver and limit the items retrieved to the first 250

#### Extract data from pages and create dataframe

In [ ]:
def extractData__data(driver, items, item_type, item_state):
    
    page_urls = []
    img_urls = []
    titles = []
    prices = []
    descs = []
    types = []
    states = []
    child_bools = []
    dates = []
    int_sizes = []
    char_sizes = []

    for item in items:
        # Get the url for the displayed image
        img_urls.append(item.find_element(By.TAG_NAME, 'img').get_attribute('src'))
        
        # Save the item type and state from the filter parameter
        types.append(item_type)
        states.append(item_state)

        # Click on item
        item.click()

        # Switch to new opened tab
        driver.switch_to.window(driver.window_handles[1])
        
        # Get the url for the item
        page_urls.append(driver.current_url)

        try:
            # Wait for the item details to appear
            item_details = WebDriverWait(
                driver, 30).until(
                    EC.visibility_of_element_located((By.CLASS_NAME, 'detail-item'))
                )
        except:
            
            # Close new tab
            driver.close()

            # Switch back to initial tab
            driver.switch_to.window(driver.window_handles[0])
            
            titles.append(item.find_element(By.CLASS_NAME, 'ItemCard__title').text)
            
            price = item.find_element(By.CLASS_NAME, 'ItemCard__price').text
            price = price.replace('.', '').replace(',', '.').replace('€', '')
            prices.append(float(price.strip()))
            
            descs.append(item.find_element(By.CLASS_NAME, 'ItemCard__description').text)
            
            child_bools.append(None)
            dates.append(None)
            int_sizes.append(None)
            char_sizes.append(None)
            continue

        # Get the title
        title = driver.find_element(By.ID, 'item-detail-title').text
        titles.append(title)

        # Get the price
        price = driver.find_element(By.CLASS_NAME, 'card-product-detail-price').text
        price = price.replace('.', '').replace(',', '.').replace('€', '').replace('EUR', '')
        prices.append(float(price.strip()))

        # Get the description for the item
        description = driver.find_element(By.CLASS_NAME, 'card-product-detail-description').text
        descs.append(description)

        # Assign default value of False to child
        child = False

        # Check description and title for each of the provided keywords
        child_keywords = ["niño/a", "niño", "niña", "niños", "niñas", "niño/as"]

        for word in child_keywords:
            if (word in title) or (word in description):
                child = True

        child_bools.append(child)
        
        # Get the bike size
        text = driver.find_element(By.CLASS_NAME, 'container-detail').text
        text = text.replace('\n', ' ')

        int_size, char_size = None, None

        if re.search(r'talla [0-9]+', text.lower()):
            # Extract unique text that contains talla and any possible numeric indication of the size of the bike
            sizes = set(re.findall(r'talla [0-9]+', text.lower())) | set(re.findall(r'talla[0-9]+', text.lower()))
            # Remove talla and clear all spaces from the found text so we keep only the numeric size
            sizes = map(lambda x: x.strip('talla').strip(), sizes)
            # Separate sizes with '/' if there's more than one different size
            int_size = '/'.join(set(sizes).difference({''}))

        if re.search(r'talla [a-z]+', text.lower()):
            # Extract unique text that contains talla and any possible character indication of the size of the bike
            sizes = set(re.findall(r'talla [a-z]+', text.lower())) | set(re.findall(r'talla[a-z]+', text.lower()))
            # Remove talla and clear all spaces from the found text so we keep only the character sizes
            sizes = map(lambda x: x.strip('talla').strip(), sizes)
            # Limit results to predefined result set
            sizes = set(sizes).intersection({'xs', 's', 'peque', 'm', 'median', 'l', 'grande'})
            # Separate sizes with '/' if there's more than one different size
            char_size = '/'.join(set(sizes).difference({''}))

        int_sizes.append(int_size)
        char_sizes.append(char_size)

        # Get the publication date
        date = driver.find_element(By.CLASS_NAME, 'card-product-detail-user-stats-published').text

        dates.append(date)

        # Close new tab
        driver.close()

        # Switch back to initial tab
        driver.switch_to.window(driver.window_handles[0])
        
    df = pd.DataFrame({
        'Link': page_urls, 
        'Title': titles,
        'Description': descs,
        'Price': prices,
        'Image': img_urls,
        'Type': types,
        'State': states,
        'Children': child_bools,
        'Size': int_sizes,
        'Size (letter)': char_sizes,
        'Date': dates
    })
    
    return driver, df

In [ ]:
def extractData(driver, item_type, item_state):
        
    # Prepare page/data for extraction
    driver, items = extractData__preprocess(driver)
    
    driver.execute_script('window.scrollBy(0, 500);')
    WebDriverWait(driver, 30)
    
    item_group = driver.find_element(By.CLASS_NAME, 'ItemCardList')
    num_of_items = len(item_group.find_elements(By.TAG_NAME, 'a'))
    
    # Check if new elements now appear on the page and if it increases the results but stays under 250
    if (num_of_items > len(items)):
        if (len(items) < 250):
            # Extracting the items again to include the new items
            extractData(driver, item_type, item_state)
    
    # Extract data
    driver, df = extractData__data(driver, items, item_type, item_state)
    
    return driver, df

#### Create combinations of subcategories and run all required functions

In [ ]:
item_types = ['Bicicletas de carretera', 'Bicicletas plegables', 'MTB']
item_states = ['Nuevo', 'Como nuevo', 'En buen estado']

# Create a complete combination of the above filters
combinations = list(product(item_types, item_states))

data = pd.DataFrame()

for item_type, item_state in combinations:
    # Open webdriver (browser)
    DRIVER_PATH = '/Downloads/chromedriver'
    driver = webdriver.Chrome(DRIVER_PATH)
    
    driver.set_window_size(width=1200, height=830)

    driver = getPage(driver)
    driver = searchItem(driver)
    driver = filterData(driver, item_type, item_state)
    driver, df = extractData(driver, item_type, item_state)
    data = pd.concat([data, df])
    print(f'{item_type}, {item_state}, {df.shape[0]}')
    
    driver.close()

In [ ]:
data.info()

#### Delete duplicated items based on the page url (link)

In [ ]:
data.drop_duplicates(subset=['Link'], inplace=True)

In [ ]:
data.info()

#### Use Spanish Locale

In [ ]:
locale.setlocale(locale.LC_TIME, 'es_ES')

#### Convert published date from string to datetime obj

In [ ]:
data['Date'] = pd.to_datetime(data['Date'], format='%d-%b-%Y')

#### Replace other identified sizes and convert the letter sizes to upper case

In [ ]:
size_replacers = {'peque': 's', 'median': 'm', 'grande': 'l'}

for key, value in size_replacers.items():
    data['Size (letter)'] = data['Size (letter)'].apply(lambda x: x.replace(key, value) if x else x)
    
data['Size (letter)'] = data['Size (letter)'].str.upper()

#### Type casting column to numeric type

In [ ]:
data['Size'] = pd.to_numeric(data['Size'], errors='coerce')

#### Replace incorrect numerically sized bikes with np.nan

In [ ]:
data.loc[data['Size'] < 46, 'Size'] = np.nan

#### Replace any null values with np.nan

In [ ]:
data.fillna(np.nan, inplace=True)

#### Replace any empty string with np.nan

In [ ]:
data.replace({'': np.nan}, inplace=True)

#### Final dataset

In [ ]:
agg = data.groupby(['Type', 'State'], as_index=False)['Price'].mean()

In [ ]:
agg